In [ ]:
my_tif = 'BlueMtHearthforShawnJeff.tif'

In [ ]:
#https://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html#clip-a-geotiff-with-shapefile
import gdal

# open dataset
ds = gdal.Open('BlueMtHearthforShawnJeff.tif')
cols = ds.RasterXSize
rows = ds.RasterYSize
bands = ds.RasterCount
print(cols, rows, bands)
ds = None

In [1]:
from osgeo import ogr

daShapefile = r"CharcoalHearthsforJeffShawn.shp"

dataSource = ogr.Open(daShapefile)
daLayer = dataSource.GetLayer(0)
layerDefinition = daLayer.GetLayerDefn()


for i in range(layerDefinition.GetFieldCount()):
    print(layerDefinition.GetFieldDefn(i).GetName())
print("Name  -  Type  Width  Precision")
for i in range(layerDefinition.GetFieldCount()):
    fieldName =  layerDefinition.GetFieldDefn(i).GetName()
    fieldTypeCode = layerDefinition.GetFieldDefn(i).GetType()
    fieldType = layerDefinition.GetFieldDefn(i).GetFieldTypeName(fieldTypeCode)
    fieldWidth = layerDefinition.GetFieldDefn(i).GetWidth()
    GetPrecision = layerDefinition.GetFieldDefn(i).GetPrecision()

    print(fieldName + " - " + fieldType+ " " + str(fieldWidth) + " " + str(GetPrecision))

AttributeError: 'NoneType' object has no attribute 'GetLayer'

In [ ]:
#Thanks
#https://podaac.jpl.nasa.gov/forum/viewtopic.php?f=5&t=657
    
import os
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from osgeo import gdal
from osgeo import osr
import numpy as np
import math

#Plot setup
fig= plt.figure(figsize=(10,8))

ax = plt.subplot(111,aspect = 'equal')
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0, hspace=0)

#Map setup
map = Basemap(resolution='f', projection='cyl', llcrnrlon=-74.72, llcrnrlat=39.71, urcrnrlon=-80.53, urcrnrlat=41.18)

parallels = np.arange(-50,50,1.)
meridians = np.arange(0,360,1)

map.drawparallels(parallels,labels=[1,0,0,0],color='w', fontsize=10, fontweight='bold')
meri = map.drawmeridians(meridians,labels=[0,0,0,1],color='w', fontsize=10, fontweight='bold')

#Load colormap and setup elevation contour levels
cmap=plt.get_cmap("jet")
clevs = np.linspace(0, 3000, 21)

# Load GeoTiff data
raster = my_tif
ds = gdal.Open(raster)

#Get the dimentions of column and row
nc   = ds.RasterXSize
nr   = ds.RasterYSize

#Read elevation data
data = ds.ReadAsArray()

#Get Longitude and Latitude info
geotransform = ds.GetGeoTransform()
xOrigin      = geotransform[0]
yOrigin      = geotransform[3]
pixelWidth   = geotransform[1]
pixelHeight  = geotransform[5]

#Generate Longitude and Latitude array
lons = xOrigin + np.arange(0, nc)*pixelWidth
lats = yOrigin + np.arange(0, nr)*pixelHeight

#Contour plot
x, y = map(*np.meshgrid(lons, lats))

cs=map.contourf(x, y, data, clevs, cmap=cmap)

map.drawparallels(parallels,labels=[1,0,0,0],color='k', fontsize=10, fontweight='bold')
meri = map.drawmeridians(meridians,labels=[0,0,0,1],color='k', fontsize=10, fontweight='bold')

cb = map.colorbar(cs, 'bottom', size='5%', pad='10%')

cb.set_label('Elevation (m)', fontsize=12, fontweight='bold')
cb.ax.tick_params(labelsize=10)

plt.show()

In [ ]:
import georaster
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

fig = plt.figure(figsize=(8,8))

m = Basemap(projection='lcc', lon_0=67.5, lat_0=-68.5, \
            height=950000, width=580000, resolution='h')

m.drawcoastlines()
m.fillcontinents(color='beige')

image = georaster.SingleBandRaster( my_tif, latlon=False)
plt.imshow(image.r, extent=image.extent, zorder=10, alpha=0.6)

plt.show()

In [ ]:

from mpl_toolkits.basemap import Basemap
from mpl_toolkits.basemap import shiftgrid
import matplotlib.pyplot as plt
from osgeo import gdal
import numpy as np

-80.53, 39.71, -74.72, 41.18

map = Basemap(resolution='f', projection='cyl', llcrnrlon=-74.72, llcrnrlat=39.71, urcrnrlon=-80.53, urcrnrlat=41.18)
#map = Basemap(projection='tmerc', 
#              lat_0=0, lon_0=3,
#              llcrnrlon=74, 
#              llcrnrlat=39, 
#              urcrnrlon=81, 
#              urcrnrlat=43)

ds = gdal.Open(my_tif)
elevation = ds.ReadAsArray()
print(elevation)
#map.imshow(plt.imread('../sample_files/orthophoto.jpg'))

map.imshow(elevation, cmap = plt.get_cmap('Blues'), alpha = 0.5)

plt.show()

In [ ]:
import xarray as xr
from affine import Affine
da = xr.open_rasterio(my_tif)
print (da)
transform = Affine.from_gdal(*da.attrs['transform']) # this is important to retain the geographic attributes from the file

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(111)
ax.imshow(da.variable.data[0])
plt.show()

In [ ]:
print(da.variable.data[0])

In [ ]:
#https://medium.com/the-barometer/read-and-plot-geotiff-in-python-xarray-cartopy-fd48faf1c503
# Imports
import xarray as xr
from affine import Affine
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as mticker

# Read the data
da = xr.open_rasterio(my_tif)
transform = Affine.from_gdal(*da.attrs['transform'])

print(da.attrs['transform'])
# Define extents
#llcrnrlon=-74.72, llcrnrlat=39.71, urcrnrlon=-80.53, urcrnrlat=41.18
lat_min = 39.71
lat_max = 41.18
lon_min = 74.72
lon_max = 80.53

# Define the projection
crs=ccrs.PlateCarree()

In [ ]:

fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(111)
ax.imshow(da.variable.data[0])
plt.show()

In [ ]:

# Create meshgrid from geotiff
nx, ny = da.sizes['x'], da.sizes['y']
x, y = np.meshgrid(np.arange(nx), np.arange(ny)) * transform
print(da.sizes)
print(transform)
# Plot!
fig = plt.figure(figsize=(32,8))
#fig = plt.figure()
ax = fig.add_subplot(111, projection=crs)
#ax.coastlines(resolution='10m', alpha=0.1)
ax.contourf(x, y, da.variable.data[0], cmap='Greys')
ax.set_extent([lon_min, lon_max, lat_min, lat_max])

# Grid and Labels
gl = ax.gridlines(crs=crs, draw_labels=True, alpha=0.5)
gl.xlabels_top = None
gl.ylabels_right = None
xgrid = np.arange(lon_min-0.5, lon_max+.5, 1.)
ygrid = np.arange(lat_min, lat_max+1, 1.)
gl.xlocator = mticker.FixedLocator(xgrid.tolist())
gl.ylocator = mticker.FixedLocator(ygrid.tolist())
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 14, 'color': 'black'}
gl.ylabel_style = {'size': 14, 'color': 'black'}
plt.show()

In [ ]:
#https://ocefpaf.github.io/python4oceanographers/blog/2015/03/02/geotiff/
from osgeo import gdal, osr

gdal.UseExceptions()


fname = my_tif

ds = gdal.Open(fname)
data = ds.ReadAsArray()
gt = ds.GetGeoTransform()
proj = ds.GetProjection()

inproj = osr.SpatialReference()
inproj.ImportFromWkt(proj)

print(inproj)

In [ ]:
import cartopy.crs as ccrs


projcs = inproj.GetAuthorityCode('PROJCS')
projection = ccrs.epsg(projcs)
print(projection)

In [ ]:
import matplotlib.pyplot as plt

subplot_kw = dict(projection=projection)
fig, ax = plt.subplots(figsize=(9, 9), subplot_kw=subplot_kw)

extent = (gt[0], gt[0] + ds.RasterXSize * gt[1],
          gt[3] + ds.RasterYSize * gt[5], gt[3])

img = ax.imshow(data[:3, :].transpose((1,0)), extent=extent, origin='upper')

In [ ]:
data[3,1]

In [ ]:
#Thanksfrom osgeo import gdal

filepath = r"LandsatData/LC81910182016153LGN00_sr_band4.tif"

# Open the file:
raster = gdal.Open(filepath)

# Check type of the variable 'raster'
type(raster)
#https://gis.stackexchange.com/questions/182489/find-rectangle-around-point-with-python
from math import sqrt,atan,pi
import pyproj
geod = pyproj.Geod(ellps='WGS84')

width = 10000. # m
height = 20000. # m
rect_diag = sqrt( width**2 + height**2 )

center_lon = -78.6389
center_lat = 35.7806

azimuth1 = atan(width/height)
azimuth2 = atan(-width/height)
azimuth3 = atan(width/height)+pi # first point + 180 degrees
azimuth4 = atan(-width/height)+pi # second point + 180 degrees

pt1_lon, pt1_lat, _ = geod.fwd(center_lon, center_lat, azimuth1*180/pi, rect_diag)
pt2_lon, pt2_lat, _ = geod.fwd(center_lon, center_lat, azimuth2*180/pi, rect_diag)
pt3_lon, pt3_lat, _ = geod.fwd(center_lon, center_lat, azimuth3*180/pi, rect_diag)
pt4_lon, pt4_lat, _ = geod.fwd(center_lon, center_lat, azimuth4*180/pi, rect_diag)

wkt_point = 'POINT (%.6f %.6f)' % (center_lon, center_lat)
wkt_poly = 'POLYGON (( %.6f %.6f, %.6f %.6f, %.6f %.6f, %.6f %.6f, %.6f %.6f ))' % (pt1_lon, pt1_lat, pt2_lon, pt2_lat, pt3_lon, pt3_lat, pt4_lon, pt4_lat, pt1_lon, pt1_lat)

In [ ]:
https://automating-gis-processes.github.io/2016/Lesson7-read-raster.html
from osgeo import gdal

filepath = my_tif

# Open the file:
raster = gdal.Open(filepath)

# Check type of the variable 'raster'
type(raster)

In [ ]:
# Projection
raster.GetProjection()

# Dimensions
raster.RasterXSize
raster.RasterYSize

# Number of bands
raster.RasterCount

# Metadata for the raster dataset
raster.GetMetadata()

In [ ]:
# Read the raster band as separate variable
band = raster.GetRasterBand(1)

# Check type of the variable 'band'
type(band)

# Data type of the values
gdal.GetDataTypeName(band.DataType)

In [ ]:
# Compute statistics if needed
if band.GetMinimum() is None or band.GetMaximum()is None:
    band.ComputeStatistics(0)
    print("Statistics computed.")

# Fetch metadata for the band
band.GetMetadata()

# Print only selected metadata:
print ("[ NO DATA VALUE ] = ", band.GetNoDataValue()) # none
print ("[ MIN ] = ", band.GetMinimum())
print ("[ MAX ] = ", band.GetMaximum())

In [ ]:
# Read raster data as numeric array from GDAL Dataset
rasterArray = raster.ReadAsArray()

In [ ]:
#Check the datatype of variables
type(rasterArray)
type(raster)

In [ ]:
from osgeo import gdal_array

# Read raster data as numeric array from file
rasterArray = gdal_array.LoadFile(filepath)

In [ ]:
rasterArray.min()

In [ ]:
import numpy as np

# Get nodata value from the GDAL band object
nodata = band.GetNoDataValue()

#Create a masked array for making calculations without nodata values
rasterArray = np.ma.masked_equal(rasterArray, nodata)
type(rasterArray)

# Check again array statistics
rasterArray.min()

In [ ]:
# Import necessary modules
In [1]: import geopandas as gpd

# Set filepath (fix path relative to yours)
In [2]: fp = "CharcoalHearthsforJeffShawn.shp"

# Read file using gpd.read_file()
In [3]: data = gpd.read_file(fp)

In [ ]:
In [4]: type(data)

In [ ]:
In [5]: data.head()

In [ ]:
In [6]: data.plot();

In [ ]:
In [7]: data.crs

In [ ]:
In [8]: data['geometry'].head()

In [ ]:
from shapely.geometry import Point, Polygon
# Create a Polygon
lat_min = 39.71
lat_max = 41.18
lon_min = -74.72
lon_max = -80.53

(-75.71725268214374, 40.764397146754874, 0.0)
(-75.71695514265, 40.774222278709765, 0.0)
(-75.70005461271417, 40.76409483152501, 0.0)
(-75.69975455768758, 40.773919920041344, 0.0)



coords = [(lon_min, lat_min), (lon_max, lat_min), (lon_max, lat_max), (lon_min, lat_max)]
poly = Polygon(coords)

selection = data[0:]
for index, row in selection.iterrows():
    pt = row['geometry']
    print(pt, pt.within(poly))


In [ ]:
from osgeo import gdal
ds = gdal.Open(my_tif)
width = ds.RasterXSize
height = ds.RasterYSize
gt = ds.GetGeoTransform()
minx = gt[0]
miny = gt[3] + width*gt[4] + height*gt[5] 
maxx = gt[0] + width*gt[1] + height*gt[2]
maxy = gt[3] 

In [ ]:
print(minx,maxx,miny,maxy)

In [ ]:
def wgs84_to_modissinu ( lon, lat ):
    wgs84 = osr.SpatialReference( )
    wgs84.ImportFromEPSG( 4326 )
    modis_sinu = osr.SpatialReference() 
    modis_sinu.ImportFromProj4 ( "+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs" )
    tx = osr.CoordinateTransformation(wgs84, modis_sinu)
    modis_x, modis_y, modis_z = tx.TransformPoint ( lon, lat )
    return modis_x, modis_y, modis_z 


In [ ]:
***** Move it down!!!

In [ ]:
This is being re-written above 


# https://gis.stackexchange.com/questions/129847/obtain-coordinates-and-corresponding-pixel-values-from-geotiff-using-python-gdal
#    #open raster layer
src_ds=gdal.Open(my_tif) 
gt=src_ds.GetGeoTransform()

print("gt",gt)
rb=src_ds.GetRasterBand(1)

#Get the dimentions of column and row
nc   = src_ds.RasterXSize
nr   = src_ds.RasterYSize

xOrigin      = gt[0]
yOrigin      = gt[3]
pixelWidth   = gt[1]
pixelHeight  = gt[5]

#Generate Longitude and Latitude array
lons = xOrigin + np.arange(0, nc)*pixelWidth
lats = yOrigin + np.arange(0, nr)*pixelHeight


gdal.UseExceptions() #so it doesn't print to screen everytime point is outside grid
print(src_ds)






# Define the source projection, WGS84 lat/lon. 
nad83_1 = osr.SpatialReference( ) # Define a SpatialReference object
nad83_1.ImportFromEPSG(7019) # And set it to WGS84 using the EPSG code

# Now for the target projection, MODIS sinusoidal
modis_sinu = osr.SpatialReference() # define the SpatialReference object
# In this case, we get the projection from a Proj4 string
modis_sinu.ImportFromProj4 ("+proj=sinu +R=6371007.181 +nadgrids=@null +wktext")

# Now, we define a coordinate transformtion object, *from* wgs84 *to* modis_sinu
tx = osr.CoordinateTransformation( nad83_1, modis_sinu )

#lyr=src_ds.GetLayer(0)

#for feat in lyr:
#    geom=feat.GetGeometryRef()
#    mx=geom.Centroid().GetX()
#    my=geom.Centroid().GetY()

#    px = int((mx - gt[0]) / gt[1]) #x pixel
#    py = int((my - gt[3]) / gt[5]) #y pixel
        
        
# Make annotation file for the jpeg from polygons
for index, row in hearth_data_poly.iterrows():
    poly = row['geometry']
    #print(row, row['id'])
    print(poly)
    
    boundary = poly.bounds
    print(boundary[0])
    print(boundary[1])
    print(boundary[2])
    print(boundary[3])
    pt1_coords = [(boundary[0], boundary[1])]
    pt2_coords = [(boundary[2], boundary[3])]
    pt1 = Point(pt1_coords)
    print(pt1)
    lon = boundary[0]
    lat = boundary[1]
    #lon = float ( lon )
    #lat = float ( lat )
    #lon = -4.025
    #lat= 56.59
    # Actually do the transformation using the TransformPoint method
    print("lon,lat",lon, lat)
    modis_x, modis_y,modis_z = wgs84_to_modissinu ( lon,lat )    
    
    print("origin",xOrigin, yOrigin)
    print("modis",modis_x, modis_y)
    #https://jgomezdans.github.io/gdal_notes/first_steps.html
    pixel_x = (modis_x - xOrigin)*pixelWidth    
    
    # The difference in distance between the UL corner (geot[0] \
    # and point of interest. Scaled by geot[1] to get pixel number
    pixel_y = (modis_y - yOrigin)*pixelHeight
    #In [55]: pixel_x
    #Out[55]: 2132.115490094644 # A real number, not an integer!
    print("pixels",pixel_x, pixel_y)
    #In [59]: pixel_y = (5936117.4 - geot[3])/(geot[5]) # Like for pixel_x, \
        #but in vertical direction. Note the different elements of geot \
        #being used
    
    
    #for i in range(0, poly.GetPointCount()):
    # GetPoint returns a tuple not a Geometry
#        pt = poly.GetPoint(i)
 #       print ("%i). POINT (%d %d)" %(i, pt[0], pt[1]))
    #select only points inside the bounds of the geotiff AND are confirmed to be hearths
    #if(pt.within(poly)==True):
#        if(row['Confirmed']=='hearth'):
 #           print(pt, pt.within(poly))

# Save a shapefile with just the points inside the geotiff and confirmed hearths

In [131]:
# Import necessary modules
import geopandas as gpd

# Set filepath (fix path relative to yours)
fp = "/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Shapefiles_5-20-2020/Charcoal-Hearths.shp"

# Read file using gpd.read_file()
data = gpd.read_file(fp)
    

#https://stackoverflow.com/questions/2922532/obtain-latitude-and-longitude-from-a-geotiff-file
from osgeo import osr, gdal

# get the existing coordinate system
ds = gdal.Open("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/39/39_jeff_slope.tif")

old_cs= osr.SpatialReference()

old_cs.ImportFromWkt(ds.GetProjectionRef())

# create the new coordinate system
nad83_wkt = """
    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPHEROID["GRS 1980",6378137,298.257222101,
                AUTHORITY["EPSG","7019"]],
            TOWGS84[0,0,0,0,0,0,0],
            AUTHORITY["EPSG","6269"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4269"]]"""


#GEOGCS["WGS 84",
#    DATUM["WGS_1984",
#        SPHEROID["WGS 84",6378137,298.257223563,
#            AUTHORITY["EPSG","7030"]],
#        AUTHORITY["EPSG","6326"]],
#    PRIMEM["Greenwich",0,
#        AUTHORITY["EPSG","8901"]],
#    UNIT["degree",0.01745329251994328,
#        AUTHORITY["EPSG","9122"]],
#    AUTHORITY["EPSG","4326"]]"""


new_cs = osr.SpatialReference()
#new_cs .ImportFromWkt(wgs84_wkt)
new_cs .ImportFromWkt(nad83_wkt)

# create a transform object to convert between coordinate systems
transform = osr.CoordinateTransformation(old_cs,new_cs) 

#get the point to transform, pixel (0,0) in this case
width = ds.RasterXSize
height = ds.RasterYSize
gt = ds.GetGeoTransform()
minx = gt[0]
miny = gt[3] + width*gt[4] + height*gt[5] 
maxx = gt[0] + width*gt[1] + height*gt[2]
maxy = gt[3]

#get the coordinates in lat long
latlong1 = transform.TransformPoint(minx,miny)
print(latlong1)
latlong2 = transform.TransformPoint(minx,maxy)
print(latlong2)
latlong3 = transform.TransformPoint(maxx,maxy)
print(latlong3)
latlong4 = transform.TransformPoint(maxx,miny)
print(latlong4)

(-80.03467167100652, 41.21431609733511, 0.0)
(-80.04043856209015, 41.37894262196131, 0.0)
(-79.8218581677315, 41.38308947736159, 0.0)
(-79.81664118024932, 41.218452607697024, 0.0)


In [1]:
import geopandas as gpd
#https://stackoverflow.com/questions/2922532/obtain-latitude-and-longitude-from-a-geotiff-file
from osgeo import osr, gdal

def get_poly_with_lat_long_from_geotif(geotif_fp):
    # Import necessary modules


    # get the existing coordinate system
    ds = gdal.Open(geotif_fp)

    old_cs= osr.SpatialReference()

    old_cs.ImportFromWkt(ds.GetProjectionRef())

    # create the new coordinate system
    nad83_wkt = """
    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPHEROID["GRS 1980",6378137,298.257222101,
                AUTHORITY["EPSG","7019"]],
            TOWGS84[0,0,0,0,0,0,0],
            AUTHORITY["EPSG","6269"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4269"]]"""

    new_cs = osr.SpatialReference()

    new_cs .ImportFromWkt(nad83_wkt)

    # create a transform object to convert between coordinate systems
    transform = osr.CoordinateTransformation(old_cs,new_cs) 

    #get the point to transform, pixel (0,0) in this case
    width = ds.RasterXSize
    height = ds.RasterYSize
    gt = ds.GetGeoTransform()
    minx = gt[0]
    miny = gt[3] + width*gt[4] + height*gt[5] 
    maxx = gt[0] + width*gt[1] + height*gt[2]
    maxy = gt[3]

    #get the coordinates in lat long
    latlong1 = transform.TransformPoint(minx,miny)
    print(latlong1)
    latlong2 = transform.TransformPoint(minx,maxy)
    print(latlong2)
    latlong3 = transform.TransformPoint(maxx,maxy)
    print(latlong3)
    latlong4 = transform.TransformPoint(maxx,miny)
    print(latlong4)
    
    # Create an empty geopandas GeoDataFrame
    polydata = gpd.GeoDataFrame()
    polydata['geometry'] = None

    # Create a Polygon
    coords = [(latlong1[0], latlong1[1]), (latlong2[0], latlong2[1]), (latlong3[0], latlong3[1]), (latlong4[0], latlong4[1])]
    poly = Polygon(coords)
    #polydata.loc[0, 'geometry'] = poly
    #polydata.loc[0, 'id'] = 0
    #print(polydata)
    #newdata = newdata.append(poly, ignore_index=True)
    # Determine the output path for the Shapefile
    #outfp = r"/storage/images/charcoal_hearth_hill/images/slope/atestof_just_this_geotiff_border.shp"

    # Write the data into that Shapefile
    #polydata.to_file(outfp)
    return(poly)

In [ ]:
get_poly_with_lat_long_from_geotif("/storage/images/charcoal_hearth_hill/images/slope/39_slope.tif")

In [2]:
# Import necessary modules first
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import fiona




# Create an empty geopandas GeoDataFrame
polydata = gpd.GeoDataFrame()
polydata['geometry'] = None

# Create a Polygon
coords = [(latlong1[0], latlong1[1]), (latlong2[0], latlong2[1]), (latlong3[0], latlong3[1]), (latlong4[0], latlong4[1])]
poly = Polygon(coords)
polydata.loc[0, 'geometry'] = poly
polydata.loc[0, 'id'] = 0
print(polydata)
#newdata = newdata.append(poly, ignore_index=True)
# Determine the output path for the Shapefile
outfp = r"/storage/images/charcoal_hearth_hill/images/slope/just_this_geotiff_border.shp"

# Write the data into that Shapefile
polydata.to_file(outfp)

print("1")
# Create an empty geopandas GeoDataFrame
hearth_data = gpd.GeoDataFrame()
hearth_data['geometry'] = None

nohearth_data = gpd.GeoDataFrame()
nohearth_data['geometry'] = None

# Create an empty geopandas GeoDataFrame
hearth_data_poly = gpd.GeoDataFrame()
hearth_data_poly['geometry'] = None


selection = data[0:]

for index, row in selection.iterrows():
    pt = row['geometry']
    #print(row)
    #select only points inside the bounds of the geotiff AND are confirmed to be hearths
    if(pt.within(poly)==True):
        if(row['Confirmed']=='hearth'):
            print(pt, pt.within(poly))
            hearth_data = hearth_data.append(row, ignore_index=True)
            
            polygon_half_size = 0.0000895078
            polygon_half_size = 0.0001745402
            #get the coordinates in lat long
            # pp_ = point_poly - the polygon around the points
            pp_minx=pt.bounds[0]-polygon_half_size
            print(pt.bounds[0])
            print(pp_minx)
            pp_maxx=pt.bounds[0]+polygon_half_size
            pp_miny=pt.bounds[1]-polygon_half_size
            pp_maxy=pt.bounds[1]+polygon_half_size
            
            pp_latlong1 = transform.TransformPoint(pp_minx,pp_miny)
            #print(latlong1)
            pp_latlong2 = transform.TransformPoint(pp_minx,pp_maxy)
            #print(latlong2)
            pp_latlong3 = transform.TransformPoint(pp_maxx,pp_maxy)
            #print(latlong3)
            pp_latlong4 = transform.TransformPoint(pp_maxx,pp_miny)
            #print(latlong4)
            # Create a Polygon
            pp_coords = [(pp_minx, pp_miny), (pp_minx, pp_maxy), (pp_maxx, pp_maxy), (pp_maxx, pp_miny)]
            #pp_coords = [(pp_minx, pp_latlong1[1]), (pp_minx, pp_latlong2[1]), (pp_latlong3[0], pp_latlong3[1]), (pp_latlong4[0], pp_latlong4[1])]
            print("pp_coords")
            print(pp_coords)
            ppoly = Polygon(pp_coords)
            new_pp_row = {'id':row['id'], 'geometry':ppoly}
            #hearth_data_poly.loc[0, 'geometry'] = ppoly
            #hearth_data_poly.loc[0, 'id'] = row['id']
            hearth_data_poly = hearth_data_poly.append(new_pp_row, ignore_index=True)
        if(row['Confirmed']=='nohearth'):
            print(pt, pt.within(poly))
            nohearth_data = nohearth_data.append(row, ignore_index=True)

            
#print("-----------------------------------")
#print(latlong1,latlong4)
#selection = data[0:]
#for index, row in selection.iterrows():
#    pt = row['geometry']
#    if(pt.bounds[0]>=latlong1[0] and pt.bounds[0]<=latlong4[0] and pt.bounds[1]>=latlong1[1] and pt.bounds[1]<=latlong4[1]):
#        print(pt, pt.within(poly))
#        print(pt.bounds[0])
#        #newdata = newdata.append(row, ignore_index=True)

NameError: name 'latlong1' is not defined

In [ ]:
print("*** hearth_data")
print(hearth_data)
print("*** nohearth_data")
print(nohearth_data)
print("*** hearth_data_poly")
print(hearth_data_poly)

In [ ]:
# Determine the output path for the Shapefile
outfp = r"/storage/images/charcoal_hearth_hill/images/slope/this_geotiff_hearth.shp"
# Write the data into that Shapefile
hearth_data.to_file(outfp)


# Determine the output path for the Shapefile
outfp = r"/storage/images/charcoal_hearth_hill/images/slope/this_geotiff_nohearth.shp"
# Write the data into that Shapefile
nohearth_data.to_file(outfp)


# Determine the output path for the Shapefile
outfp = r"/storage/images/charcoal_hearth_hill/images/slope/this_geotiff_hearth_poly.shp"
# Write the data into that Shapefile
hearth_data_poly.to_file(outfp)

# Save .tif as .jpg

In [ ]:

import numpy as np
ds = gdal.Open(my_tif)
#geoTrans = srcImage.GetGeoTransform()

band = ds.GetRasterBand(1)
width = ds.RasterXSize
height = ds.RasterYSize

data = band.ReadAsArray(0, 0, width, height)
#convert all the bad data
data[data==-9999.0] = 0
max_value = numpy.max(data)
color_multiplier = 255/(max_value-1)
print(color_multiplier)
data = data*color_multiplier
print(data)
data_int = np.array(data, dtype='int')
print(data_int)
#clip = ds.readasarray(ds)
data_int = data_int.astype(gdalnumeric.uint8)

gdalnumeric.SaveArray(data_int, "BlueMtHearthforShawnJeff.jpg", format="JPEG")

# Needed for below

In [3]:
import numpy as np

def tif_to_jpg(tif_file,output_file_name, output_folder):
    #print(tif_file,output_file_name, output_folder)
    ds = gdal.Open(tif_file)
    #geoTrans = srcImage.GetGeoTransform()

    band = ds.GetRasterBand(1)
    width = ds.RasterXSize
    height = ds.RasterYSize

    data = band.ReadAsArray(0, 0, width, height)
    #convert all the bad data
    data[data==-9999.0] = 0
    max_value = numpy.max(data)
    color_multiplier = 255/(max_value-1)
    #print(color_multiplier)
    data = data*color_multiplier
    #print(data)
    data_int = np.array(data, dtype='int')
    #print(data_int)
    #clip = ds.readasarray(ds)
    data_int = data_int.astype(gdalnumeric.numpy.uint8)

    gdalnumeric.SaveArray(data_int, output_folder+output_file_name+".jpg", format="JPEG")

#https://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html?highlight=rasterize

In [4]:
# Run this
def write_annot(obj_type, obj_annots_dir, obj_f_num, org_f_path,org_f_width,org_f_height,org_f_depth, refPts):
    # With credit to: 
    # https://www.geeksforgeeks.org/reading-writing-text-files-python/
    annot_file_path = obj_annots_dir+obj_f_num+'.xml'
    annot_file = open(annot_file_path,"w") 

    annot_file.write("<annotation>\n") 
    annot_file.write("	<folder>"+obj_type+"</folder>\n") 
    annot_file.write("	<filename>"+obj_f_num+".jpg</filename>\n") 
    annot_file.write("	<path>"+org_f_path+"</path>\n") 
    annot_file.write("	<source>\n") 
    annot_file.write("		<database>Muhlenberg_charcoal_hearths</database>\n") 
    annot_file.write("	</source>\n") 
    annot_file.write("	<size>\n") 
    annot_file.write("		<width>"+str(org_f_width)+"</width>\n") 
    annot_file.write("		<height>"+str(org_f_height)+"</height>\n") 
    annot_file.write("		<depth>"+str(org_f_depth)+"</depth>\n") 
    annot_file.write("	</size>\n") 
    annot_file.write("	<segmented>0</segmented>\n") 
    for ocn in range(0,len(refPts)):
        refPt = refPts[ocn]
        refPtMin = refPt[0]
        refPtMax = refPt[1]
        
        #Sometimes the mouse gets dragged from the bottom to the top, etc
        if(refPtMin[1]>refPtMax[1]):
            ytemp = refPtMin[1]
            refPtMin[1] = refPtMax[1]
            refPtMax[1]=ytemp
        if(refPtMin[0]>refPtMax[0]):
            xtemp = refPtMin[0]
            refPtMin[0] = refPtMax[0]
            refPtMax[0]=xtemp 
        
        annot_file.write("	<object>\n") 
        annot_file.write("		<name>"+obj_type+"</name>\n") 
        annot_file.write("		<number>"+str(ocn)+"</number>\n") 
        annot_file.write("		<truncated>0</truncated>\n") 
        annot_file.write("		<difficult>0</difficult>\n") 
        annot_file.write("		<bndbox>\n") 
        annot_file.write("			<xmin>"+str(refPtMin[0])+"</xmin>\n") 
        annot_file.write("			<ymin>"+str(refPtMin[1])+"</ymin>\n") 
        annot_file.write("			<xmax>"+str(refPtMax[0])+"</xmax>\n") 
        annot_file.write("			<ymax>"+str(refPtMax[1])+"</ymax>\n") 
        annot_file.write("		</bndbox>\n") 
        annot_file.write("	</object>\n") 
    annot_file.write("</annotation>\n") 
    annot_file.close()

In [135]:
# note below the Proj4 incantation.
# The words of this spell come from the EPSG definition, visible for the CRS in QGIS
#NAD83 / Pennsylvania South
#Extent
#-80.53, 39.71, -74.72, 41.18
#Proj4
#+proj=lcc +lat_1=40.96666666666667 +lat_2=39.93333333333333 +lat_0=39.33333333333334 +lon_0=-77.75 +x_0=600000 +y_0=0 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs

#See and Thanks to:
#https://geog0111.readthedocs.io/en/latest/Chapter7_FittingPhenologyModels.html?highlight=Proj4#selecting-data-from-a-raster-file

def convert_coordinates(x_location, y_location,
                        #src_transform={'EPSG':32128},
                        #dst_transform={'Proj4':"+proj=lcc +lat_1=41.95 +lat_2=40.88333333333333 +lat_0=40.16666666666666 +lon_0=-77.75 +x_0=600000 +y_0=0 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs"}):
                        
                       #src_transform={'EPSG':4326},
                        #dst_transform={'Proj4':"+proj=longlat +datum=WGS84 +no_defs"}):
                       #dst_transform={'Proj4':"+proj=lcc +lat_1=40.96666666666667 +lat_2=39.93333333333333 +lat_0=39.33333333333334 +lon_0=-77.75 +x_0=600000 +y_0=0 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs"}):
    src_transform={'EPSG':32128},
    dst_transform={'Proj4':"+proj=lcc +lat_1=41.95 +lat_2=40.88333333333333 +lat_0=40.16666666666666 +lon_0=-77.75 +x_0=600000 +y_0=0 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs"}):
    """A function to convert coordinates from one target coordinate
    representation to another. The input an output transformation can be given
    in either EPSG codes or Proj4 strings, by providing the function with a
    dictionary with the desired convention as a key, and with the relevant
    codes as its only element.

    Parameters
    ----------
    x_location: float
        The x location
    y_location: float
        The y location
    src_transform: dict
        A dictionary with keys either "EPSG" or "Proj4" (anything else throws
        an exception) with the description of the **input** projection
    dst_transform: dict
        A dictionary with keys either "EPSG" or "Proj4" (anything else throws
        an exception) with the description of the **output** projection
    Returns
    --------

    The transformed x and y coordinates"""
    input_coords = osr.SpatialReference()
    # In this case, we use EPSG code
    try:
        input_coords.ImportFromEPSG(src_transform["EPSG"])
    except KeyError:
        input_coords.ImportFromProj4(src_transform["Proj4"])
    except KeyError:
        raise ValueError("src_transform not dictionary with EPSG/Proj4 keys!")


    output_coords = osr.SpatialReference()
    try:
        output_coords.ImportFromEPSG(dst_transform["EPSG"])
    except KeyError:
        output_coords.ImportFromProj4(dst_transform["Proj4"])
    except KeyError:
        raise ValueError("src_transform not dictionary with EPSG/Proj4 keys!")


    transformation = osr.CoordinateTransformation(input_coords,
                                                 output_coords)
    output_x, output_y, output_z = transformation.TransformPoint(x_location,
                                                         y_location)
    return output_x, output_y


##################################################################
# Test function
##################################################################

#x_location, y_location = 43.3623, -8.4115 # In WGS84

#x_location, y_location = -75.7128140898394, 40.771356587287

#print (convert_coordinates(x_location, y_location))

In [136]:
import gdal
def get_pixel(raster, point_x, point_y):
    """Get the pixel for given coordinates (in the raster's convention, not
    checked!) for a raster file.

    Parameters
    ----------
    raster: string
        A GDAL-friendly raster filename
    point_x: float
        The Easting in the same coordinates as the raster (not checked!)
    point_y: float
        The Northing in the same coordinates as the raster (not checked!)

    Returns
    -------
    The row/column (or column/row, depending on how you define it)
    """
    g = gdal.Open(raster)
    if g is None:
        raise ValueError(f"{raster:s} cannot be opened!")
    geoT = g.GetGeoTransform()
    inv_geoT = gdal.InvGeoTransform(geoT)
    r, c = (gdal.ApplyGeoTransform(inv_geoT, point_x, point_y))
    return int(r + 0.5), int(c + 0.5)

##################################################################
# Test function
##################################################################


#fname = "/home/plewis/public_html/geog0111_data/lai_files/" + \
#              "MCD15A3H.A2016273.h17v04.006.2016278070708.hdf"
#gdal_fname = 'HDF4_EOS:EOS_GRID:"%s":MOD_Grid_MCD15A3H:Fpar_500m' % fname

#gdal_fname ="BlueMtHearthforShawnJeff.tif"
#print (get_pixel(gdal_fname, 771970.4122612458, 161656.97900192798))

In [137]:
#/storage/images/charcoal_hearth_hill/images/split_tifs/390819.tif
#src_ds.RasterXSize 1024 768
#boundary = poly.bounds
#-79.94041862482659
#41.345360655348536
#-79.94006954442658
#41.345709735748535
#lon,lat -79.94041862482659 41.345360655348536
#54 -92229
#lon,lat -79.94006954442658 41.345709735748535
#84 -92267
#boundary = poly.bounds
#-79.93842727130712
#41.34473854341196
#-79.9380781909071
#41.345087623811956


x_location, y_location = -79.94041862482659, 41.345360655348536

print (convert_coordinates(x_location, y_location))

gdal_fname ="/storage/images/charcoal_hearth_hill/images/split_tifs/390819.tif"
cx,cy=convert_coordinates(x_location, y_location)
print (get_pixel(gdal_fname,cx,cy ))


(-79.94041862482659, 41.345360655348536)
(-416703, 133423)


In [138]:
#Modified version - uses a raster already open to save processing
def get_pixel_with_geot(geoT, point_x, point_y):
    """Get the pixel for given coordinates (in the raster's convention, not
    checked!) for a raster file.

    Parameters
    ----------
    geoT: 
        raster.GetGeoTransform()
    point_x: float
        The Easting in the same coordinates as the raster (not checked!)
    point_y: float
        The Northing in the same coordinates as the raster (not checked!)

    Returns
    -------
    The row/column (or column/row, depending on how you define it)
    """
    
    inv_geoT = gdal.InvGeoTransform(geoT)
    r, c = (gdal.ApplyGeoTransform(inv_geoT, point_x, point_y))
    return int(r + 0.5), int(c + 0.5)

In [ ]:
# https://gis.stackexchange.com/questions/129847/obtain-coordinates-and-corresponding-pixel-values-from-geotiff-using-python-gdal
#    #open raster layer
src_ds=gdal.Open(my_tif) 
gt=src_ds.GetGeoTransform()
#Get the dimentions of column and row
nc   = src_ds.RasterXSize
nr   = src_ds.RasterYSize
print(nc,nr)
gdal.UseExceptions() #so it doesn't print to screen everytime point is outside grid

refPts = []      
refPt = []
#I'm hoping the tifs will be 1452X1091        
# Make annotation file for the jpeg from polygons
for index, row in hearth_data_poly.iterrows():
    poly = row['geometry']
    #print(row, row['id'])
    #print(poly)
    
    boundary = poly.bounds
    #print(boundary[0])
    #print(boundary[1])
    #print(boundary[2])
    #print(boundary[3])

    lon = boundary[0]
    lat = boundary[1]
    
    print("lon,lat",lon, lat)
        
    cx,cy=convert_coordinates(lon, lat)
    p1x,p1y= (get_pixel_with_geot(gt,cx,cy ))
    
    
    print(p1x,p1y)
    lon = boundary[2]
    lat = boundary[3]
    
    print("lon,lat",lon, lat)
        
    cx,cy=convert_coordinates(lon, lat)
    p2x,p2y= (get_pixel_with_geot(gt,cx,cy ))
    print(p2x,p2y)
    #p2y is smaller than p1y so we'll exchange them to keep p1 as the minimum.
    ptemp=p1y
    p1y=p2y
    p2y=ptemp
    if(p1x<0):
        p1x=0
    if(p1y<0):
        p1y=0
    if(p2x<0):
        p2x=0
    if(p2y<0):
        p2y=0

    if(p1x>nc):
        p1x=nc
    if(p1y>nr):
        p1y=nr
    if(p2x>nc):
        p2x=nc
    if(p2y>nr):
        p2y=nr
     
        
    refPt = [(p1x, p1y)]
    refPt.append((p2x, p2y))
    refPts.append(refPt)

object_type = 'charcoal_hearth_hill'
object_annots_dir = 'C:\\a_orgs\\carleton\\hist3814\\R\\charcoalhearths_data\\'+object_type+'\\annots\\'
write_annot(object_type, object_annots_dir, "BlueMtHearthforShawnJeff", "BlueMtHearthforShawnJeff.jpg",nc,nr,1, refPts)    

In [ ]:
#https://gis.stackexchange.com/questions/92207/split-a-large-geotiff-into-smaller-regions-with-python-and-gdal
    

In [5]:
# Thanks to
#https://gis.stackexchange.com/questions/92207/split-a-large-geotiff-into-smaller-regions-with-python-and-gdal
import os  
import numpy
#import gdal
from osgeo import gdal, osr
import math
from itertools import chain
#import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import numpy as np
import gdalnumeric

def get_extent(dataset):

    cols = dataset.RasterXSize
    rows = dataset.RasterYSize
    transform = dataset.GetGeoTransform()
    minx = transform[0]
    maxx = transform[0] + cols * transform[1] + rows * transform[2]

    miny = transform[3] + cols * transform[4] + rows * transform[5]
    maxy = transform[3]

    return {
            "minX": str(minx), "maxX": str(maxx),
            "minY": str(miny), "maxY": str(maxy),
            "cols": str(cols), "rows": str(rows)
            }

def create_tiles(minx, miny, maxx, maxy, nw, nh):
    width = maxx - minx
    height = maxy - miny

    matrix = []

    for j in range(nw, 0, -1):
        for i in range(0, nh):

            ulx = minx + (width/nw) * i # 10/5 * 1
            uly = miny + (height/nh) * j # 10/5 * 1

            lrx = minx + (width/nw) * (i + 1)
            lry = miny + (height/nh) * (j - 1)
            matrix.append([[ulx, uly], [lrx, lry]])

    return matrix



def create_standard_tiles(minx, miny, maxx, maxy):
    print("standard",minx, miny, maxx, maxy)
    width = maxx - minx
    height = maxy - miny
    
    standard_width = 1024
    standard_height = 768
    
    
    nw = int(width/standard_width)+1
    nh = int(height/standard_height)+1
    print(nw,nh)
    #print(nw*standard_width,nh*standard_height)
    
    matrix = []

    for j in range(nh, 0, -1):
        for i in range(0, nw):

            ulx = minx + (standard_width * i) # 10/5 * 1
            uly = miny + (standard_height * j) # 10/5 * 1

            lrx = minx + (standard_width * (i + 1))
            lry = miny + (standard_height * (j - 1))
            if(lrx>maxx):
                lrx=maxx
            if(lrx<minx):
                lrx=minx
              
            if(lry>maxy):
                lry=maxy
            if(lry<miny):
                lry=miny

            if(ulx>maxx):
                ulx=maxx
            if(ulx<minx):
                ulx=minx

            if(uly>maxy):
                uly=maxy
            if(uly<miny):
                uly=miny

            #print("matrix:",ulx, uly,lrx, lry)
            matrix.append([[ulx, uly], [lrx, lry],[i,j]])
    print("standard",minx, miny, maxx, maxy)
    return matrix


def split(file_name):
    raw_file_name = os.path.splitext(os.path.basename(file_name))[0].replace("_downsample", "")
    driver = gdal.GetDriverByName('GTiff')
    dataset = gdal.Open(file_name)
    
    
    proj_dataset = osr.SpatialReference(wkt=dataset.GetProjection())
    print(proj_dataset.GetAttrValue('AUTHORITY',1))
    print(proj_dataset)
    
    
    band = dataset.GetRasterBand(1)
    transform = dataset.GetGeoTransform()


    extent = get_extent(dataset)

    cols = int(extent["cols"])
    rows = int(extent["rows"])

    print ("Columns: ", cols)
    print ("Rows: ", rows)

    minx = float(extent["minX"])
    maxx = float(extent["maxX"])
    miny = float(extent["minY"])
    maxy = float(extent["maxY"])
    
    width = int(maxx - minx)
    height = int(maxy - miny)

    nw = int(width/1024)+1
    nh = int(height/768)+1
    
    #******
    # make a polygon for the area and see what points are in the whole area.
    #get the coordinates in lat long

    # Create an empty geopandas GeoDataFrame
    area_hearth_data = gpd.GeoDataFrame()
    area_hearth_data['geometry'] = None
    area_poly=get_poly_with_lat_long_from_geotif(file_name)
    # Read file using gpd.read_file()
    all_hearth_data = gpd.read_file("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Shapefiles_5-20-2020/Charcoal-Hearths.shp")
    selection = all_hearth_data[0:]
    #print(selection)
    for index, row in selection.iterrows():
        #print("for index, row in selection.iterrows():")
        pt = row['geometry']
        #print(row)
        #select only points inside the bounds of the geotiff AND are confirmed to be hearths
        if(pt.within(area_poly)==True):
            #print("if(pt.within(tif_poly)==True):")
            area_hearth_data = area_hearth_data.append(row, ignore_index=True)
    
    # Determine the output path for the Shapefile
    outfp = r"area_"+str(raw_file_name[:2])+"_hearth.shp"
    # Write the data into that Shapefile
    area_hearth_data.to_file(outfp)
    #*************************8
    output_path = os.path.join("data", raw_file_name)
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    #print ("GCD", math.gcd(round(width, 0), round(height, 0)))
    #print ("Width", width)
    #print ("height", height)

    # Polygons to reflect the tiles
    # Create an empty geopandas GeoDataFrame
    tile_boundary_poly = gpd.GeoDataFrame()
    tile_boundary_poly['geometry'] = None

    #tiles = create_tiles(minx, miny, maxx, maxy, nw,nh)
    tiles = create_standard_tiles(minx, miny, maxx, maxy)
    transform = dataset.GetGeoTransform()
    xOrigin = transform[0]
    yOrigin = transform[3]
    pixelWidth = transform[1]
    pixelHeight = -transform[5]

    #print (xOrigin, yOrigin)

    srs = osr.SpatialReference()
    srs.ImportFromEPSG(32128)
    #srs.ImportFromEPSG(4326)
    #dst_ds.SetProjection( srs.ExportToWkt() )



    
    tile_num = 0
    for tile in tiles:
        #print (tile)

        minx = tile[0][0]
        maxx = tile[1][0]
        miny = tile[1][1]
        maxy = tile[0][1]
        tilex = "00"+str(tile[2][0])
        tilex = tilex[-2:]
        tiley = "00"+str(tile[2][1])
        tiley = tiley[-2:]
        
        
        
        p1 = (minx, maxy)
        p2 = (maxx, miny)

        i1 = int((p1[0] - xOrigin) / pixelWidth)
        j1 = int((yOrigin - p1[1])  / pixelHeight)
        i2 = int((p2[0] - xOrigin) / pixelWidth)
        j2 = int((yOrigin - p2[1]) / pixelHeight)

        #print (i1, j1)
        #print (i2, j2)

        new_cols = i2-i1
        new_rows = j2-j1

        data = band.ReadAsArray(i1, j1, new_cols, new_rows)
        #print("data",data.mean(),data)
        #if len(list(set(chain(*data))))<3:
        if(np.all(data==data[0])):
            print("")
            #print("data.mean",data.mean())
        else:
            #print("data.mean.too",data.mean())
            
            new_x = xOrigin + i1*pixelWidth
            new_y = yOrigin - j1*pixelHeight

            #print (new_x, new_y)

            new_transform = (new_x, transform[1], transform[2], new_y, transform[4], transform[5])
            output_file_name = raw_file_name[:2] + tilex + tiley
            output_file_base = output_file_name + ".tif"
            #output_file = os.path.join("/storage/images/charcoal_hearth_hill/images/split_tifs", raw_file_name, output_file_base)
            #output_file = os.path.join("/storage/images/charcoal_hearth_hill/images/split_tifs", raw_file_name, output_file_base)
            output_file = "/storage/images/charcoal_hearth_hill/images/split_tifs/"+output_file_base

            dst_ds = driver.Create(output_file,
                                   new_cols,
                                   new_rows,
                                   1,
                                   gdal.GDT_Float32)

            #writing output raster
            dst_ds.GetRasterBand(1).WriteArray( data )
            dst_ds.SetProjection( srs.ExportToWkt() )
            dst_ds.SetGeoTransform(new_transform)
            tif_metadata = {
                "minX": str(minx), "maxX": str(maxx),
                "minY": str(miny), "maxY": str(maxy)
            }
            dst_ds.SetMetadata(tif_metadata)

            
#path = r"/storage/images/charcoal_hearth_hill/images/slope/39_slope.tif"
#d = gdal.Open(path)
#proj = osr.SpatialReference(wkt=d.GetProjection())
#print(proj.GetAttrValue('AUTHORITY',1))
#print(proj)

            #setting extension of output raster
            # top left x, w-e pixel resolution, rotation, top left y, rotation, n-s pixel resolution
            #dst_ds.SetGeoTransform(new_transform)

            #wkt = dataset.GetProjection()

            # setting spatial reference of output raster
            #srs = osr.SpatialReference()
            #srs.ImportFromWkt(wkt)
            #dst_ds.SetProjection( srs.ExportToWkt() )

            dst_ds = None            

            #make a polygon for this tile
            tpoly = get_poly_with_lat_long_from_geotif(output_file)
            new_tp_row = {'id':tile_num, 'geometry':tpoly}
            tile_boundary_poly = tile_boundary_poly.append(new_tp_row, ignore_index=True)
            
            #Save .jpg
            # close tif before re-opening it and saving as jpg
            # The jpg is to be consumed by the model.
            tif_to_jpg(output_file,output_file_name,"/storage/images/charcoal_hearth_hill/images/jpgs/")
            #/storage/images/charcoal_hearth_hill/images_training/
            #Are there points inside the geotiff?
            #annotate_tif_if_it_has_hearths("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Shapefiles_5-20-2020/Charcoal-Hearths.shp",tpoly,output_file,output_file_name)
            annotate_tif_if_it_has_hearths(area_hearth_data,tpoly,output_file,output_file_name)
            
            #We need to refine to handle points on the border
            
        #Close output raster dataset
        dst_ds = None

        tile_num += 1

    dataset = None
    
    # Determine the output path for the Shapefile
    outfp = r"a_tile_boundary.shp"
    # Write the data into that Shapefile
    tile_boundary_poly.to_file(outfp)
    #print(tile_boundary_poly)


In [10]:
import gdal
from osgeo import ogr
def get_pixel_with_geot(geoT, point_x, point_y):
    """Get the pixel for given coordinates (in the raster's convention, not
    checked!) for a raster file.

    Parameters
    ----------
    raster: string
        A GDAL-friendly raster filename
    point_x: float
        The Easting in the same coordinates as the raster (not checked!)
    point_y: float
        The Northing in the same coordinates as the raster (not checked!)

    Returns
    -------
    The row/column (or column/row, depending on how you define it)
    """
    #g = gdal.Open(raster)
    #if g is None:
    #    raise ValueError(f"{raster:s} cannot be opened!")
    #geoT = open_raster.GetGeoTransform()
    inv_geoT = gdal.InvGeoTransform(geoT)
    r, c = (gdal.ApplyGeoTransform(inv_geoT, point_x, point_y))
    return int(r + 0.5), int(c + 0.5)

def annotate_tif_if_it_has_hearths(area_hearth_data, tif_poly, tif_fp, annot_file_name):
    #print("annotate_tif_if_it_has_hearths",area_hearth_data, tif_poly, tif_fp, annot_file_name)
    # Set filepath (fix path relative to yours)
    #fp = "/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Shapefiles_5-20-2020/Charcoal-Hearths.shp"


    # Create an empty geopandas GeoDataFrame
    hearth_data = gpd.GeoDataFrame()
    hearth_data['geometry'] = None
    hearth_data.crs = {'init':'epsg:4326'}

    # Create an empty geopandas GeoDataFrame
    hearth_data_poly = gpd.GeoDataFrame()
    hearth_data_poly['geometry'] = None
    hearth_data_poly.crs = {'init':'epsg:4326'}

    selection = area_hearth_data[0:]
    point_count=0
    
    print(tif_fp)
    src_ds=gdal.Open(tif_fp) 
    gt=src_ds.GetGeoTransform()
    
    #old_cs= osr.SpatialReference()
    #old_cs.ImportFromWkt(src_ds.GetProjectionRef())

    # create the new coordinate system
    #nad83_wkt = """
    #GEOGCS["NAD83",
    #    DATUM["North_American_Datum_1983",
    #        SPHEROID["GRS 1980",6378137,298.257222101,
    #            AUTHORITY["EPSG","7019"]],
    #        TOWGS84[0,0,0,0,0,0,0],
    #        AUTHORITY["EPSG","6269"]],
    #    PRIMEM["Greenwich",0,
    #        AUTHORITY["EPSG","8901"]],
    #    UNIT["degree",0.0174532925199433,
    #        AUTHORITY["EPSG","9122"]],
    #    AUTHORITY["EPSG","4269"]]"""

    #new_cs = osr.SpatialReference()
    #new_cs .ImportFromWkt(nad83_wkt)

    # create a transform object to convert between coordinate systems
    #transform = osr.CoordinateTransformation(old_cs,new_cs)
    
    target = osr.SpatialReference(wkt=src_ds.GetProjection())
    source = osr.SpatialReference()
    #source.ImportFromEPSG(4326)
    source.ImportFromEPSG(32128)
    target.ImportFromEPSG(32128)
    transform = osr.CoordinateTransformation(source, target)

    for index, row in selection.iterrows():
        #print("for index, row in selection.iterrows():")
        pt = row['geometry']
        #print(row)
        #select only points inside the bounds of the geotiff AND are confirmed to be hearths
        if(pt.within(tif_poly)==True):
            #print("if(pt.within(tif_poly)==True):")
            point_count=point_count+1
            #print(pt, pt.within(tif_poly))
            hearth_data = hearth_data.append(row, ignore_index=True)
            
            polygon_half_size = 0.0000895078
            polygon_half_size = 0.0001745402
            #get the coordinates in lat long
            # pp_ = point_poly - the polygon around the points
            pp_minx=pt.bounds[0]-polygon_half_size
            #print(pt.bounds[0])
            #print(pp_minx)
            pp_maxx=pt.bounds[0]+polygon_half_size
            pp_miny=pt.bounds[1]-polygon_half_size
            pp_maxy=pt.bounds[1]+polygon_half_size
            
            pp_latlong1 = transform.TransformPoint(pp_minx,pp_miny)
            #print("latlong1",latlong1)
            print("pp_minx",pp_minx)
            pp_latlong2 = transform.TransformPoint(pp_minx,pp_maxy)
            #print(latlong2)
            pp_latlong3 = transform.TransformPoint(pp_maxx,pp_maxy)
            #print(latlong3)
            pp_latlong4 = transform.TransformPoint(pp_maxx,pp_miny)
            #print(latlong4)
            # Create a Polygon
            pp_coords = [(pp_minx, pp_miny), (pp_minx, pp_maxy), (pp_maxx, pp_maxy), (pp_maxx, pp_miny)]
            #pp_coords = [(pp_minx, pp_latlong1[1]), (pp_minx, pp_latlong2[1]), (pp_latlong3[0], pp_latlong3[1]), (pp_latlong4[0], pp_latlong4[1])]
            #print("pp_coords")
            #print(pp_coords)
            ppoly = Polygon(pp_coords)
            new_pp_row = {'id':row['id'], 'geometry':ppoly}
            #hearth_data_poly.loc[0, 'geometry'] = ppoly
            #hearth_data_poly.loc[0, 'id'] = row['id']
            hearth_data_poly = hearth_data_poly.append(new_pp_row, ignore_index=True)
    #Do we have points inside this tif?  If so, annotate and copy>0
    if(point_count)>0:
        
    
        hearth_data_poly = hearth_data_poly.to_crs(epsg=32128)
        hearth_data = hearth_data.to_crs(epsg=32128)
        
    
        #Write out shp files for diagnostics
        outfp_base = tif_fp[:-4]
        outfp = outfp_base+"_hearth_data_poly.shp"
        # Write the data into that Shapefile
        hearth_data_poly.to_file(outfp)
        outfp = outfp_base+"_hearth_data.shp"
        # Write the data into that Shapefile
        hearth_data.to_file(outfp)
        
        
        print("****")

        #Get the dimentions of column and row
        nc   = src_ds.RasterXSize
        nr   = src_ds.RasterYSize
        print("src_ds.RasterXSize", nc,nr)
        gdal.UseExceptions() #so it doesn't print to screen everytime point is outside grid
        
        #target = osr.SpatialReference(wkt=src_ds.GetProjection())
        #source = osr.SpatialReference()
        #source.ImportFromEPSG(4326)
        #transform = osr.CoordinateTransformation(source, target)
        
        

        refPts = []      
        refPt = []
        #I'm hoping the tifs will be 1024X768        
        # Make annotation file for the jpeg from polygons
        for index, row in hearth_data_poly.iterrows():
            poly = row['geometry']
            #print(row, row['id'])
            #print(poly)
            
            boundary = poly.bounds
            print("boundary = poly.bounds")
            print(boundary[0])
            print(boundary[1])
            print(boundary[2])
            print(boundary[3])

            lon = boundary[0]
            lat = boundary[1]
    
            print("lon,lat",lon, lat)
      
            point = ogr.Geometry(ogr.wkbPoint)
            point.AddPoint(lon, lat)
            point.Transform(transform)

            #p1x,p1y = world_to_pixel(ds.GetGeoTransform(), point.GetX(), point.GetY())
            p1x,p1y = get_pixel_with_geot(gt, point.GetX(), point.GetY())
            print("p1",point.GetX(), point.GetY(),p1x,p1y)
        
            #cx,cy=convert_coordinates(lon, lat)
            #p1x,p1y= (get_pixel_with_geot(gt,cx,cy ))
    
   

            lon = boundary[2]
            lat = boundary[3]
    
            print("lon,lat",lon, lat)
        
            #cx,cy=convert_coordinates(lon, lat)
            #p2x,p2y= (get_pixel_with_geot(gt,cx,cy ))
            point = ogr.Geometry(ogr.wkbPoint)
            point.AddPoint(lon, lat)
            point.Transform(transform)

            #p2x,p2y = world_to_pixel(ds.GetGeoTransform(), point.GetX(), point.GetY())
            p2x,p2y = get_pixel_with_geot(gt, point.GetX(), point.GetY())
            print("p2",point.GetX(), point.GetY(),p2x,p2y)

            

            #p2y is smaller than p1y so we'll exchange them to keep p1 as the minimum.
            ptemp=p1y
            p1y=p2y
            p2y=ptemp
            if(p1x<0):
                p1x=0
            if(p1y<0):
                p1y=0
            if(p2x<0):
                p2x=0
            if(p2y<0):
                p2y=0

            if(p1x>nc):
                p1x=nc
            if(p1y>nr):
                p1y=nr
            if(p2x>nc):
                p2x=nc
            if(p2y>nr):
                p2y=nr
     
        
            refPt = [(p1x, p1y)]
            refPt.append((p2x, p2y))
            refPts.append(refPt)

        object_type = 'charcoal_hearth_hill'
        object_annots_dir = '/storage/images/'+object_type+'/annots/'
        write_annot(object_type, object_annots_dir, annot_file_name, annot_file_name+".jpg",nc,nr,1, refPts)
        tif_to_jpg(tif_fp,annot_file_name,"/storage/images/charcoal_hearth_hill/images_training/")


   

In [17]:
#split(file_name, n)
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/39/39blast2demPAN_DEM.tif",10)
#split("/storage/images/charcoal_hearth_hill/images/slope/39_slope.tif")

#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/39/39blast2demPAN_Slope_DEM.tif")
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/41/41blast2demPAS_Slope_DEM.tif")
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/43/43blast2demPAS_Slope_DEM.tif")
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/46/46blast2demPAS_Slope_DEM.tif")
split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/52/52partialDEM_Slope.tif")
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/55/55blast2demPAN_Slope_DEM.tif")
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/58/58blast2demPAN_Slope_DEM.tif")
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/72/72blast2demPAN_Slope_DEM.tif")


26918
PROJCS["NAD83 / UTM zone 18N",
    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPHEROID["GRS 1980",6378137,298.257222101,
                AUTHORITY["EPSG","7019"]],
            TOWGS84[0,0,0,0,0,0,0],
            AUTHORITY["EPSG","6269"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4269"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",-75],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","26918"]]
Columns:  7500
Rows:  6000
(-75.99243820721493, 40.17362215473075, 0.0)
(-75.99322666674416, 40.227670959849085, 0.0)
(-75.90508136838083, 40.22839385099543, 0.0)
(-75.90436

(-83.01783550662262, 76.88215377802975, 0.0)
(-83.01921382350865, 76.88729062763622, 0.0)
(-82.98903550205873, 76.88770661277881, 0.0)
(-82.98766504054755, 76.88256970597305, 0.0)
/storage/images/charcoal_hearth_hill/images/split_tifs/520605.tif
(-82.98766504054755, 76.88256970597305, 0.0)
(-82.98903550205873, 76.88770661277881, 0.0)
(-82.97925085061544, 76.887840969782, 0.0)
(-82.97788293617836, 76.88270404450127, 0.0)
/storage/images/charcoal_hearth_hill/images/split_tifs/520705.tif

(-83.16725165396005, 76.8749010876157, 0.0)
(-83.1686684922195, 76.88003828718479, 0.0)
(-83.13850449852575, 76.88046616523464, 0.0)
(-83.13709550650462, 76.87532890685418, 0.0)
/storage/images/charcoal_hearth_hill/images/split_tifs/520104.tif
(-83.13709550650462, 76.87532890685418, 0.0)
(-83.13850449852575, 76.88046616523464, 0.0)
(-83.10833919588453, 76.88089165342538, 0.0)
(-83.10693805112557, 76.8757543365598, 0.0)
/storage/images/charcoal_hearth_hill/images/split_tifs/520204.tif
(-83.10693805112557,

In [ ]:
41blast2demPAS_Slope_DEM

file_name = "/storage/images/charcoal_hearth_hill/images/slope/39_slope.tif"
raw_file_name = os.path.splitext(os.path.basename(file_name))[0].replace("_downsample", "")
print(raw_file_name)
driver = gdal.GetDriverByName('GTiff')
dataset = gdal.Open(raw_file_name+".tif")
band = dataset.GetRasterBand(1)
transform = dataset.GetGeoTransform()

extent = get_extent(dataset)

In [ ]:
# 
annotate_tif_if_it_has_hearths("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Shapefiles_5-20-2020/Charcoal-Hearths.shp",tpoly)

In [ ]:
import numpy as np
import numpy
import gdalnumeric
ds = gdal.Open("/storage/images/charcoal_hearth_hill/images/split_tifs/390024.tif")
#geoTrans = srcImage.GetGeoTransform()

band = ds.GetRasterBand(1)
width = ds.RasterXSize
height = ds.RasterYSize

data = band.ReadAsArray(0, 0, width, height)
#convert all the bad data
data[data==-9999.0] = 0
max_value = numpy.max(data)
color_multiplier = 255/(max_value-1)
print(color_multiplier)
data = data*color_multiplier
print(data)
data_int = np.array(data, dtype='int')
print(data_int)
#clip = ds.readasarray(ds)
data_int = data_int.astype(gdalnumeric.numpy.uint8)

gdalnumeric.SaveArray(data_int, "BlueMtHearthforShawnJeff.jpg", format="JPEG")

In [ ]:

cx,cy=convert_coordinates(-79.94041862482659, 41.345360655348536)
print(cx,cy)

In [ ]:
#/storage/images/charcoal_hearth_hill/images/split_tifs/390819.tif
#src_ds.RasterXSize 1024 768
#boundary = poly.bounds
#-79.94041862482659
#41.345360655348536
#-79.94006954442658
#41.345709735748535
#lon,lat -79.94041862482659 41.345360655348536
#54 -92229
#lon,lat -79.94006954442658 41.345709735748535
#84 -92267
#boundary = poly.bounds
#-79.93842727130712
#41.34473854341196
#-79.9380781909071
#41.345087623811956

In [103]:
import gdal
import osr

path = r"/storage/images/charcoal_hearth_hill/images/slope/39_slope.tif"
d = gdal.Open(path)
proj = osr.SpatialReference(wkt=d.GetProjection())
print(proj.GetAttrValue('AUTHORITY',1))
print(proj)



path = r"/storage/images/charcoal_hearth_hill/images/split_tifs/390711.tif"
d = gdal.Open(path)
proj = osr.SpatialReference(wkt=d.GetProjection())
print(proj.GetAttrValue('AUTHORITY',1))
print(proj)


#src_transform={'EPSG':32128},
#dst_transform={'Proj4':"+proj=lcc +lat_1=41.95 +lat_2=40.88333333333333 +lat_0=40.16666666666666 +lon_0=-77.75 +x_0=600000 +y_0=0 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs"}):
                        
                       #src_transform={'EPSG':4326},
                       #dst_transform={'Proj4':"+proj=lcc +lat_1=40.96666666666667 +lat_2=39.93333333333333 +lat_0=39.33333333333334 +lon_0=-77.75 +x_0=600000 +y_0=0 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs"}):


32128
PROJCS["NAD83 / Pennsylvania North",
    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPHEROID["GRS 1980",6378137,298.257222101,
                AUTHORITY["EPSG","7019"]],
            TOWGS84[0,0,0,0,0,0,0],
            AUTHORITY["EPSG","6269"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4269"]],
    PROJECTION["Lambert_Conformal_Conic_2SP"],
    PARAMETER["standard_parallel_1",41.95],
    PARAMETER["standard_parallel_2",40.88333333333333],
    PARAMETER["latitude_of_origin",40.16666666666666],
    PARAMETER["central_meridian",-77.75],
    PARAMETER["false_easting",600000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["X",EAST],
    AXIS["Y",NORTH],
    AUTHORITY["EPSG","32128"]]
32128
PROJCS["NAD83 / Pennsylvania North",
    GEOGCS["NAD83",
        DATUM["North_America

In [105]:
from osgeo import osr, ogr, gdal


def world_to_pixel(geo_matrix, x, y):
    """
    Uses a gdal geomatrix (gdal.GetGeoTransform()) to calculate
    the pixel location of a geospatial coordinate
    """
    ul_x= geo_matrix[0]
    ul_y = geo_matrix[3]
    x_dist = geo_matrix[1]
    y_dist = geo_matrix[5]
    pixel = int((x - ul_x) / x_dist)
    line = -int((ul_y - y) / y_dist)
    return pixel, line

# Extract target reference from the tiff file
ds = gdal.Open("/storage/images/charcoal_hearth_hill/images/split_tifs/390711.tif")
target = osr.SpatialReference(wkt=ds.GetProjection())
source = osr.SpatialReference()
source.ImportFromEPSG(4326)
transform = osr.CoordinateTransformation(source, target)

point = ogr.Geometry(ogr.wkbPoint)
point.AddPoint(-79.94036802242027, 41.28845973774296)
point.Transform(transform)

x, y = world_to_pixel(ds.GetGeoTransform(), point.GetX(), point.GetY())
print(x, y)

945 420
